# Update Certificates  
SSLCertVerificationError


In [9]:
! pip3 install certifi


In [10]:
import os
import certifi

os.environ['SSL_CERT_FILE'] = certifi.where()


# Downlaoding the Dataset 

In [11]:
from convokit import Corpus, download
corpus = Corpus(filename=download("movie-corpus"))

No configuration file found at /Users/wajeeh/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
default_backend: mem


In [12]:
corpus.print_summary_stats()

Number of Speakers: 9035
Number of Utterances: 304713
Number of Conversations: 83097


In [28]:
# Print the first 10 utterances
for utterance in list(corpus.iter_utterances())[:10]:
    print(utterance.text)


They do not!
They do to!
I hope so.
She okay?
Let's go.
Wow
Okay -- you're gonna need to learn how to lie.
No
I'm kidding.  You know how sometimes you just become this "persona"?  And you don't know how to quit?
Like my fear of wearing pastels?


In [31]:
# Print the first 10 speakers
for speaker in list(corpus.iter_speakers())[:10]:
    print(speaker.meta)


ConvoKitMeta({'character_name': 'BIANCA', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'f', 'credit_pos': '4'})
ConvoKitMeta({'character_name': 'CAMERON', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'm', 'credit_pos': '3'})
ConvoKitMeta({'character_name': 'CHASTITY', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': '?', 'credit_pos': '?'})
ConvoKitMeta({'character_name': 'JOEY', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'm', 'credit_pos': '6'})
ConvoKitMeta({'character_name': 'KAT', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'f', 'credit_pos': '2'})
ConvoKitMeta({'character_name': 'WALTER', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': 'm', 'credit_pos': '9'})
ConvoKitMeta({'character_name': 'BRUCE', 'movie_idx': 'm0', 'movie_name': '10 things i hate about you', 'gender': '?', 'credit_pos': '?'})
ConvoKitMeta({'characte

In [32]:
# Print the first 5 conversations
for convo in list(corpus.iter_conversations())[:5]:
    print(convo.get_utterance_ids())


['L1045', 'L1044']
['L985', 'L984']
['L925', 'L924']
['L872', 'L871', 'L870']
['L869', 'L868', 'L867', 'L866']


# Perform NER with SpaCy

In [49]:
import spacy

# Load spaCy's English language model
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    # Process the text with spaCy
    doc = nlp(text)

    # Extract entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Assuming you have a corpus to iterate over
for utterance in corpus.iter_utterances():
    entities = extract_entities(utterance.text)
    utterance.meta['entities'] = [{"entity": entity, "type": entity_type} for entity, entity_type in entities]

corpus.dump("movie-corpus-with-entities")


KeyboardInterrupt: 

# Aggregate Entity Counts

In [ ]:
from collections import Counter

entity_counts = Counter()

for conversation in corpus.iter_conversations():
    for utterance in conversation.iter_utterances():
        entities = utterance.meta['entities'] if 'entities' in utterance.meta else []
        for entity in entities:
            entity_type = entity['type']
            entity_counts[entity_type] += 1


In [3]:
from flask import Flask, jsonify
import spacy

In [4]:
app = Flask(__name__)

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

In [5]:

def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

@app.route('/get_entities', methods=['GET'])
def get_entities():
    # Dummy text, replace with your Convokit data retrieval logic
    text = "Apple is looking at buying U.K. startup for $1 billion"
    entities = extract_entities(text)
    return jsonify(entities)


In [6]:
@app.route('/get_entity_network', methods=['GET'])
def get_entity_network():
    # Using a dictionary to track co-occurrence
    co_occurrence = {}

    for utterance in corpus.utterances_iter():
        doc = nlp(utterance.text)
        entities = [ent.text for ent in doc.ents if ent.label_ in ['PERSON', 'ORG', 'GPE']]  # Filter entities if needed
        for i, entity in enumerate(entities):
            for adjacent in entities[i + 1:]:
                if entity != adjacent:
                    pair = tuple(sorted([entity, adjacent]))
                    co_occurrence[pair] = co_occurrence.get(pair, 0) + 1

    # Preparing nodes and links for D3
    nodes = list({entity for pair in co_occurrence for entity in pair})
    links = [{"source": pair[0], "target": pair[1], "value": count} for pair, count in co_occurrence.items()]

    return jsonify({"nodes": [{"id": node} for node in nodes], "links": links})


In [7]:
@app.route('/get_all_entities', methods=['GET'])
def get_all_entities():

    all_entities = []
    for utterance in corpus.utterances_iter():
        doc = nlp(utterance.text)
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        all_entities.append({"utterance": utterance.text, "entities": entities})

    return jsonify(all_entities)

In [8]:
if __name__ == '__main__':
    app.run(debug=True, port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/Users/wajeeh/Library/Python/3.10/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/wajeeh/Library/Python/3.10/lib/python/site-packages/traitlets/config/application.py", line 981, in launch_instance
    app.initialize(argv)
  File "/Users/wajeeh/Library/Python/3.10/lib/python/site-packages/traitlets/config/application.py", line 110, in inner
    return method(app, *args, **kwargs)
  File "/Users/wajeeh/Library/Python/3.10/lib/python/site-packages/ipykernel/kernelapp.py", line 666, in initialize
    self.init_sockets()
  File "/Users/wajeeh/Library/Python/3.10/lib/python/site-packages/ipykernel/kernelapp.py", line 307, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/Users/wajeeh/Library/Python/3.10/lib/python/site-packages/ipykernel/kernelapp.py", l

SystemExit: 1

/Users/wajeeh/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Plotting the Distribution

In [54]:
import matplotlib.pyplot as plt

# Preparing data for plotting
labels, values = zip(*entity_counts.items())

# Creating the bar chart
plt.figure(figsize=(10, 6))
plt.bar(labels, values, color='skyblue')
plt.xlabel('Entity Types')
plt.ylabel('Frequency')
plt.title('Distribution of Entity Types in Dataset')
plt.xticks(rotation=45)
plt.show()

NameError: name 'entity_counts' is not defined